In [1]:
import glob
import json
import os
import time

from multiprocessing import Pool
import pandas as pd

# local .json file with local path info
geladas_root = os.path.dirname(os.getcwd())
local_paths_file = os.path.join(geladas_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(local_paths['overhead_functions_path'])

import koger_general_functions as kgf
import mapping_functions as kmap
from drone_movement import create_gt_segment_dicts, get_segment_drone_movement

Will throw error if using functions that require gdal.


In [2]:
observation_name = "DJI_0205"
# Pix4D inserts the name of the Pix4D project in output files
pix4d_project_name = "gelada_worked_example_DJI_0205"

In [3]:
all_segment_dicts = []

save_folder_name = "worked_examples_drone_movement"

# This is where the raw video images are saved 
frame_folders_root = os.path.join(local_paths['videos_folder'],
                                  "frames", observation_name)
frame_files = sorted(glob.glob(os.path.join(frame_folders_root, "*.jpg")))

output_folder = os.path.join(frame_folders_root, save_folder_name)

pix4d_folder = os.path.join(local_paths['pix4d_folder'], observation_name)
pmatrix_file = os.path.join(pix4d_folder, "1_initial", "params",
                            f"{pix4d_project_name}_pmatrix.txt") 
anchor_info_file = os.path.join(pix4d_folder, "map-images", "anchor_frame_info.csv")
anchor_info = kmap.get_cleaned_anchor_info(anchor_info_file, pmatrix_file)

segment_dicts = create_gt_segment_dicts(frame_files=frame_files, 
                                        anchor_info=anchor_info, 
                                        output_folder=output_folder, 
                                        save=True)
all_segment_dicts.extend(segment_dicts)

Warning, /mount/zfs-kn/gelada_project/frames/DJI_0205/worked_examples_drone_movement already exists


In [5]:
if __name__ == '__main__':
    t0 = time.time()
    with Pool(processes=8) as pool:
        pool.map(get_segment_drone_movement, all_segment_dicts)
    print('total time:', time.time() - t0)

total time: 780.7001404762268


### Check resulting drone movement estimates
Compare the map locations of tracks in the last frame of an estimated segment to the map locations of tracks in the next anchor frame. If the local drone movement between anchor frames has been fully recovered, these locations should be identical. In practice, they should be low (under a couple meters). If error is too high in a given segment, adittional anchor frames can be added to this part of the observation.

In [11]:
import gdal
import numpy as np

In [13]:
# Load extra animal track and map files for validation

# -- Previously processed animal trajectories in video space ----
tracks_file = os.path.join(local_paths['base_frames_folder'],
                          observation_name, "localizations",
                          "tracks-complete.npy")
tracks = np.load(tracks_file, allow_pickle=True)

# -- Outputs from Pix4D -----------------------------------
offset_file = os.path.join(pix4d_folder, "1_initial", "params",
                            f"worked_examples_{observation_name}_offset.xyz") 
offset = kmap.load_map_offset(offset_file)

geotif_dsm_file =  os.path.join(map_folder, "3_dsm_ortho", "1_dsm",
                                f"worked_examples_{observation_name}_dsm.tif")
dsm_gtif = gdal.Open(geotif_dsm_file)
# For converting between utm and raster coordinates
dsm_transform = dsm_gtif.GetGeoTransform()
x_origin = dsm_transform[0]
y_origin = dsm_transform[3]
pixel_width = dsm_transform[1]
pixel_height = dsm_transform[5]

geotif_dtm_file =  os.path.join(map_folder, "3_dsm_ortho", 
                                "extras", "dtm", 
                                f"worked_examples_{observation_name}_dtm.tif")
dtm_gtif = gdal.Open(geotif_dtm_file)
dtm = dtm_gtif.GetRasterBand(1).ReadAsArray()
dtm = kmap.fill_with_min(dtm)
elevation_r = cv2.resize(dtm, 
                        (dsm_gtif.RasterXSize, dsm_gtif.RasterYSize), 
                        interpolation=cv2.INTER_CUBIC)

# -- Just calculated drone movement info --------------------
glob_string = os.path.join(output_folder, "drone_movement_segment_*.npy")
segment_movement_files = sorted(glob.glob(glob_string))                                   
segment_movements = [np.load(file) for file in segment_movement_files]

glob_string = os.path.join(output_folder, "inliers_segment_*.npy")
segment_inlier_files = sorted(glob.glob(segment_inlier_string))
segment_inliers = [np.load(file) for file in segment_inlier_files] 
                                                          
# -- Miscellaneous ------------------------------------------
groundtruth_obs_indexes, _ = kmap.get_groundtruth_obs_indexes(flight_logs, 
                                                              frame_folders_root)

demo_frame_name = list(flight_logs[0].loc[flight_logs[0]['used_in_map'], 'image_name'])[0]
demo_frame_file = os.path.join(frame_folders_root, 
                               '_'.join(demo_frame_name.split('_')[:-1]), # was 5 from drone_movement_test 
                               demo_frame_name)
demo_frame = plt.imread(demo_frame_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/localizations/tracks-complete.npy'